In [ ]:
from datetime import datetime

import pandas as pd
import wfdb
import os

# Caminho base onde está a pasta "ppg-database-but-ppg-2.0.0"
base_path = r"C:\Users\sebas\Downloads\brno"
data_folder = os.path.join(base_path, "ppg-database-but-ppg-2.0.0")

# 1. Ler os registos válidos com qualidade
quality_df = pd.read_csv(os.path.join(base_path, "quality-hr-ann.csv"))
# Filtrar apenas os PPG com qualidade (coluna 'Quality' == 1)
quality_ppg_ids = [f"{int(id_)}_PPG" for id_ in quality_df[quality_df['Quality'] == 1]['ID'].values]
print(f"Número de registos PPG com qualidade: {len(quality_ppg_ids)}")

# 2. Ler os caminhos dos ficheiros PPG no RECORDS.txt
with open(os.path.join(base_path, "RECORDS.txt"), "r") as f:
    all_records = []

    for line in f:
        if line.strip() and line.strip().endswith("_PPG"):
            all_records.append(line.strip())

print(f"Número total de ficheiros PPG encontrados no RECORDS.txt: {len(all_records)}")
print("\n--- VERIFICAÇÃO DE VARIÁVEIS ---")

print(f"📌 base_path: {base_path}")
print(f"📌 data_folder: {data_folder}")

print(f"\n✅ quality_ppg_ids (total {len(quality_ppg_ids)}):")
print(quality_ppg_ids[:10])  # mostrar apenas os primeiros 10

print(f"\n✅ all_records (total {len(all_records)}):")
print(all_records[:10])  # mostrar os primeiros 10

print("\n🔎 Comparação direta entre nomes:")
for i in range(min(10, len(all_records))):
    full_record = all_records[i]
    record_name = os.path.basename(full_record)
    match = record_name in quality_ppg_ids
    print(f"  - {record_name} {'✔️ MATCH' if match else '❌'}")

print("\n--- FIM DA VERIFICAÇÃO ---")

# 3. Filtrar apenas os ficheiros PPG com qualidade
for r in all_records:
    # print("nome record: " + r.split("/")[0])
    if r.split("/")[0].strip() in quality_ppg_ids:
        print("Sim")

valid_records = [r for r in all_records if os.path.basename(r) in quality_ppg_ids]
print(f"Número de ficheiros PPG válidos: {len(valid_records)}")
# Lista para guardar os sinais
ppg_data = []

for record in valid_records:
    full_path = os.path.join(data_folder, record)  # sem extensão!
    try:
        # Lê o registo PPG usando a biblioteca wfdb
        signal, _ = wfdb.rdsamp(full_path)
        # Se tiver mais que um canal, usar apenas o primeiro
        ppg_values = signal[:, 0] if signal.ndim > 1 else signal
        ppg_data.append(ppg_values)
    except Exception as e:
        print(f"Erro ao ler {record}: {e}")

# Criar DataFrame
df = pd.DataFrame(ppg_data)

print(f"\nTotal de registos lidos: {len(df)}")
df.head()